In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import torch 
import numpy as np
import torch.nn as nn
import t3nsor as t3
from torchtext import data
from torchtext import datasets
import torch.optim as optim

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
TEXT = data.Field(tokenize='spacy', fix_length=500);
LABEL = data.LabelField(dtype=torch.float);

In [7]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [8]:
train_data, valid_data = train_data.split()

In [17]:
TEXT.build_vocab(train_data, max_size=24998)
LABEL.build_vocab(train_data)

In [18]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

In [19]:
class LSTM_Classifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()       
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):      
        embedded = self.dropout(x) 
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden.squeeze(0))

In [20]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

In [21]:
lstm_model = LSTM_Classifier(embedding_dim=EMBEDDING_DIM, 
                             hidden_dim=HIDDEN_DIM, 
                             output_dim=OUTPUT_DIM,
                             n_layers=N_LAYERS,
                             bidirectional=BIDIRECTIONAL,
                             dropout=DROPOUT)

In [22]:
INPUT_DIM

25000

In [25]:
embed_model = t3.TTEmbedding(shape=[[2, 5, 2, 5, 2, 5, 5, 5], [2, 2, 2, 2, 2, 2, 2, 1]], tt_rank=8, batch_dim_last=True)

In [26]:
model = nn.Sequential(embed_model, lstm_model)

In [27]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    
    for i, batch in enumerate(iterator):
        
        optimizer.zero_grad()
                

        predictions = model(batch.text).squeeze(1)
        
        
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        if i > len(iterator):
            break
        
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            if i > len(iterator):
                break
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.689 | Train Acc: 55.75% | Val. Loss: 0.661 | Val. Acc: 60.87% |


# Pretrained model

In [ ]:
TEXT.build_vocab(train_data, max_size=24998, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

In [ ]:
pretrained_embeddings.shape

In [26]:
tt_pretrained = t3.to_tt_matrix(pretrained_embeddings.to(device), shape=[[2, 5, 2, 5, 2, 5, 5, 5], [2, 5, 2, 1, 1, 5, 1, 1]])

In [27]:
print(tt_pretrained)

A TT-Matrix of size 25000 x 100, underlying tensorshape: [2, 5, 2, 5, 2, 5, 5, 5] x [2, 5, 2, 1, 1, 5, 1, 1], TT-ranks: [1, 4, 10, 10, 10, 10, 10, 5, 1] 
 on device 'cuda:0' 


In [42]:
embed_model_pretrained = t3.TTEmbedding(init=tt_pretrained)

In [43]:
model_pre = nn.Sequential(embed_model_pretrained, lstm_model)

In [44]:
optimizer = optim.Adam(model_pre.parameters())
criterion = nn.BCEWithLogitsLoss()

In [45]:
model_pre = model_pre.to(device)
criterion = criterion.to(device)

In [48]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model_pre, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model_pre, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

AttributeError: 'Tensor' object has no attribute 'flip'

In [ ]:
import torch